<a href="https://colab.research.google.com/github/GiorgosMaragkopoulos/Quantum-data-encoding-using-QAE-PQAE/blob/main/QAE_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Autoencoders (QAEs) for latent space encoding of the Iris dataset vs linear autoencoders

In this jupyter notebook, I present the implementation of a Quantum autoencoder (QAE), which is essentially an autoencoder with a quantum feature map in the bottleneck. There will be a comparison between the performance of the model in terms of the reconstruction error with the QAE encoding VS without it (same structure, without the quantum feature map). The dataset in this example is going to be the Iris dataset, $l^2$-normalized per row.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,MinMaxScaler , Normalizer
from torch.nn.functional import normalize


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True  # may slow down training.
    torch.backends.cudnn.benchmark = False

# Example usage
set_seed(3)


# Load Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensor before normalization
normalizer = Normalizer(norm='l2')
X = normalizer.fit_transform(X)


# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 3)  # 4 inputs to 3 nodes (bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 4),
            nn.Linear(4,4)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Instantiate the model, define loss function and optimizer
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Training the autoencoder
num_epochs = 2000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, X)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/2000], Loss: 0.0563
Epoch [200/2000], Loss: 0.0239
Epoch [300/2000], Loss: 0.0214
Epoch [400/2000], Loss: 0.0208
Epoch [500/2000], Loss: 0.0204
Epoch [600/2000], Loss: 0.0195
Epoch [700/2000], Loss: 0.0153
Epoch [800/2000], Loss: 0.0070
Epoch [900/2000], Loss: 0.0036
Epoch [1000/2000], Loss: 0.0033
Epoch [1100/2000], Loss: 0.0032
Epoch [1200/2000], Loss: 0.0032
Epoch [1300/2000], Loss: 0.0031
Epoch [1400/2000], Loss: 0.0031
Epoch [1500/2000], Loss: 0.0032
Epoch [1600/2000], Loss: 0.0031
Epoch [1700/2000], Loss: 0.0031
Epoch [1800/2000], Loss: 0.0031
Epoch [1900/2000], Loss: 0.0031
Epoch [2000/2000], Loss: 0.0031


# The Quantum Autoencoder starts with an input layer of 4 neurons and passes to the bottleneck which is a hidden layer of 3 neurons. These 3 neurons are used as coefficients to the generator, and 2 complex numbers form 4 real numbers (2 real numbers and 2 imaginary numbers), which are then decoded to the original 4 dimensional output.

In [ ]:
class Quantum_Autoencoder(nn.Module):
    def __init__(self):
        super(Quantum_Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 3)  # 4 inputs to 3 nodes (bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 4)  # 3 nodes to 4 outputs
        )


        self.q0 = torch.tensor([[1], [0]], dtype=torch.cfloat)

        # Define the Pauli matrices
        self.sigma2 = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64)
        self.sigma3 = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex64)
        self.sigma4 = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64)



        # Collect the Gell-Mann matrices in a list
        self.generators = [ self.sigma2, self.sigma3, self.sigma4 ]



    def forward(self, x):
        x = self.encoder(x)

        logits = torch.empty(150, 4, dtype=torch.float)

        for i in range(150):
            encoded = torch.zeros([2,2], dtype=torch.cfloat)
            for index in range(3):
                encoded += (1j  *  x[i][index]*  self.generators[index%3])


            Exp_matrix = torch.matrix_exp(encoded )
            qubit_1 = self.q0


            qubit_1 = torch.matmul(Exp_matrix, qubit_1).T

            real_part_1 = qubit_1.real.squeeze()  # Extract the real part and remove any unnecessary dimensions
            imaginary_part_1 = qubit_1.imag.squeeze()  # Extract the imaginary part and remove any unnecessary dimensions

            # Concatenate qutrit_6_elements with squared_elements
            qubit_4_elements = torch.cat((real_part_1, imaginary_part_1  ) , dim=0)


            logits[i] =  qubit_4_elements


        x = self.decoder(logits)

        return x




# Instantiate the model, define loss function and optimizer
qae = Quantum_Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(qae.parameters(), lr=0.001)

# Training the autoencoder
num_epochs = 2000
for epoch in range(num_epochs):
    # Forward pass
    outputs = qae(X)
    loss = criterion(outputs, X)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/2000], Loss: 0.1420
Epoch [200/2000], Loss: 0.0833
Epoch [300/2000], Loss: 0.0566
Epoch [400/2000], Loss: 0.0345
Epoch [500/2000], Loss: 0.0171
Epoch [600/2000], Loss: 0.0086
Epoch [700/2000], Loss: 0.0052
Epoch [800/2000], Loss: 0.0038
Epoch [900/2000], Loss: 0.0030
Epoch [1000/2000], Loss: 0.0024
Epoch [1100/2000], Loss: 0.0019
Epoch [1200/2000], Loss: 0.0015
Epoch [1300/2000], Loss: 0.0014
Epoch [1400/2000], Loss: 0.0013
Epoch [1500/2000], Loss: 0.0012
Epoch [1600/2000], Loss: 0.0012
Epoch [1700/2000], Loss: 0.0012
Epoch [1800/2000], Loss: 0.0012
Epoch [1900/2000], Loss: 0.0012
Epoch [2000/2000], Loss: 0.0012


The reconstruction loss is 3 times lower in the quantum autoencoder, in contrast to the purely classical version.